In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import make_moons
from sklearn.preprocessing import StandardScaler

BELLS AND WHISTLES

In [2]:
X,y=make_moons()

In [3]:
X_bias=np.c_[np.ones((X.shape[0],1)),X,
             np.square(X[:,0]),np.square(X[:,1]),
            X[:,0]**3,X[:,1]**3]

m,n=X_bias.shape

y=y.reshape(-1,1)

np.random.seed(42)
arr=np.arange(m)
np.random.shuffle(arr)

test_ratio=0.2
test_size=int(m*test_ratio)

X_train=X_bias[:-test_size]
X_test=X_bias[-test_size:]

y_train=y[:-test_size]
y_test=y[-test_size:]

In [4]:
def feed_batch(epoch,batch_index,batch_size):
    
    np.random.seed(epoch*batch_size)
    arr=np.arange(m)
    
    np.random.shuffle(arr)
    indices=[batch_index*batch_size,min((batch_index+1)*batch_size,m)]
    
    X_batch=X_train[indices[0]:indices[1]]
    y_batch=y_train[indices[0]:indices[1]]
    
    return X_batch,y_batch

In [5]:
X=tf.placeholder(tf.float32,shape=(None,n),name='X')
y=tf.placeholder(tf.float32,shape=(None,1),name='y')

In [6]:
def logistic_regression(X,y,initializer=None, seed=42, learning_rate=0.01):
    n_inputs_including_bias=int(X.get_shape()[1])
    with tf.name_scope('logistic_regression'):
        with tf.name_scope('model'):
            if initializer is None:
                initializer=tf.random_uniform([n_inputs_including_bias,1],-1,1,seed=seed)
            theta=tf.Variable(initializer,name='theta')
            logits=tf.matmul(X,theta,name='logits')
            y_proba=tf.sigmoid(logits)
        with tf.name_scope('train'):
            loss=tf.losses.log_loss(y,y_proba,scope='loss')
            optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
            training_op=optimizer.minimize(loss)
            loss_summary=tf.summary.scalar('log_loss',loss)
        with tf.name_scope('init'):
            init=tf.global_variables_initializer()
        with tf.name_scope('save'):
            saver=tf.train.Saver()
    return y_proba, loss, training_op, loss_summary, init, saver

In [7]:
from datetime import datetime

def log_dir(prefix=''):
    now=datetime.utcnow().strftime('%Y%m%d%H%M%S')
    root_logdir='tf_logs'
    if prefix:
        prefix+= '-'
    name=prefix+'run-'+now
    return "{}/{}/".format(root_logdir,name)

In [8]:
logdir=log_dir('logreg')

y_proba, loss, training_op, loss_summary, init, saver=logistic_regression(X,y)

file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

In [12]:
import os 

n_epochs=10001
batch_size=16
n_batches=int(np.ceil(m/batch_size))

checkpoint_path='/tmp/my_logreg_model.ckpt'
checkpoint_epoch_path=checkpoint_path+'.epoch'
final_model_path='./my_logreg_model'

with tf.Session() as sess:
#     if os.path.isfile(checkpoint_epoch_path):
#         with open(checkpoint_epoch_path,'rb') as f:
#             start_epoch=int(f.read())
#         print('Training was interrupted. Continuing at epoch',start_epoch)
#         saver.restore(sess,checkpoint_path)
#     else:
    start_epoch=0
    sess.run(init)
        
    for epoch in range(start_epoch,n_epochs):
        for batch_index in range(n_batches):
            X_batch,y_batch=feed_batch(epoch,batch_index,batch_size)
            sess.run(training_op, feed_dict={X:X_batch,y:y_batch})
        loss_val, summary_str=sess.run([loss,loss_summary],feed_dict={X:X_test,y:y_test})
        file_writer.add_summary(summary_str,epoch)
        
        if epoch%500==0:
            print("Epoch:", epoch,"\tLoss:",loss_val)
            saver.save(sess,checkpoint_path)
            with open(checkpoint_epoch_path,'wb') as f:
                f.write(b'%d'%(epoch+1))
    y_proba_val=y_proba.eval(feed_dict={X:X_test,y:y_test})

Epoch: 0 	Loss: 0.997331
Epoch: 500 	Loss: 0.321336
Epoch: 1000 	Loss: 0.266861
Epoch: 1500 	Loss: 0.231768
Epoch: 2000 	Loss: 0.204833
Epoch: 2500 	Loss: 0.183205
Epoch: 3000 	Loss: 0.165467
Epoch: 3500 	Loss: 0.150696
Epoch: 4000 	Loss: 0.138239
Epoch: 4500 	Loss: 0.127612
Epoch: 5000 	Loss: 0.118454
Epoch: 5500 	Loss: 0.110487
Epoch: 6000 	Loss: 0.1035
Epoch: 6500 	Loss: 0.0973223
Epoch: 7000 	Loss: 0.0918243
Epoch: 7500 	Loss: 0.0869018
Epoch: 8000 	Loss: 0.0824699
Epoch: 8500 	Loss: 0.0784599
Epoch: 9000 	Loss: 0.0748142
Epoch: 9500 	Loss: 0.0714844
Epoch: 10000 	Loss: 0.0684325


In [7]:
1+1

2